In [1]:
%%bash
pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl size=4438492 sha256=d76a8da30ed95cb9af3d78742b6d710e1c0613f09313e74ded1a220d0e510d6b
  Stored in directory: /root/.cache/pip/wheels/f3/35/48/0b9a7076995eea5ea64a7e4bc3f0f342f453080795276264e7
Successfully built mpi4py


In [30]:
%%writefile mpi_program1.py
import numpy as np
import pandas as pd
import sklearn
from pathlib import Path
from sklearn.cluster import KMeans
from mpi4py import MPI
from sklearn.metrics.pairwise import euclidean_distances

if __name__ == '__main__':
  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  size = comm.Get_size()
  dest = 0
  result = np.zeros(1)
  min = np.zeros(1)
  data_path = "/content/DS_2019_public.csv"
  xName = "KWHCOL"
  yName = "BTUELCOL"
  df = pd.read_csv(data_path, encoding='latin1')
  data = df[[xName, yName]].values
  np.random.shuffle(data)
  
  if rank == 0:
    data[:, 0] = data[:, 0] / df[xName].max()
    data[:, 1] = data[:, 1] / df[yName].max()
  
  kmeans = KMeans(n_clusters=rank + 2)
  labels = kmeans.fit_predict(data)
  print(f"For {rank + 2} clusters centers is:", kmeans.cluster_centers_)

  ss_w = kmeans.inertia_
  dists = euclidean_distances(kmeans.cluster_centers_)
  tri_dists = dists[np.triu_indices(rank + 2, 1)]
  tri_dists_sq = tri_dists ** 2
  ss_b = sum(tri_dists_sq)

  #RS индекс
  index = round (ss_b/(ss_b + ss_w), 3)
  comm.Isend(index, dest)
  
  results = comm.gather(index, dest)
  if rank == dest:
    print("RS indexes for 2 to 8 clasters:", results)
  pass

Overwriting mpi_program1.py


In [34]:
%%bash
mpiexec --allow-run-as-root -np 8 python mpi_program1.py

For 2 clusters centers is: [[0.10236137 0.10236129]
 [0.01413818 0.01413817]]
For 3 clusters centers is: [[ 4077.5151365  13912.48085914]
 [  616.30758957  2102.84156634]
 [10815.10391086 36901.12720613]]
For 5 clusters centers is: [[  355.72644198  1213.73879025]
 [ 8540.92088249 29141.61263549]
 [ 4673.33146369 15945.40626018]
 [ 2145.98868141  7322.11331534]
 [17219.74661972 58753.77498592]]
For 4 clusters centers is: [[  504.68112233  1721.97220869]
 [ 3172.98952159 10826.2407631 ]
 [ 7156.99514663 24419.65956803]
 [16195.09631111 55257.6597    ]]For 6 clusters centers is: [[   356.93169951   1217.85111738]
 [  8316.66017483  28376.43576923]
 [  2137.97638964   7294.77531566]
 [  4614.92608953  15746.12771143]
 [ 16006.13731707  54612.92996341]
 [ 60995.431      208116.552     ]]

For 7 clusters centers is: [[   239.23407079    816.26667697]
 [ 16489.92502778  56263.61693056]
 [  5422.36582305  18501.10827707]
 [  3152.95258089  10757.87560546]
 [  9035.98291746  30830.76719683]
 [

sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
sys:1: DtypeWarning: Columns (30,56,96,114) have mixed types.Specify dtype option on import or set low_memory=False.
